## Imports

In [1]:
# importing all relevant libraries, packages, and functionalities

import pandas as pd
import re
from textacy import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.parsing.preprocessing import remove_stopwords
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
# installing NLTK data for lemmatization

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
# importing the data set

df = pd.read_excel(r'Artificial Intelligence Companies.xlsx')

## Data Preparation 

In [ ]:
# dropping all companies where Full Description is NaN and resetting index

df_clean = df[df['Full Description'].notna()].reset_index()

In [ ]:
# filtering the categories column of the data frame for specific, relevant categories

categories = ['Health', 'Care', 'Diagnostics', 'Biotechnology', 'Hospital', 'Medical', 'Pharmaceutical', 'Genetics']

df_clean['Category incl.'] = df_clean['Categories'].apply(lambda row: any(category in row for category in categories))
df_clean = df_clean.loc[df_clean['Category incl.'] == True].reset_index().drop(columns=['index', 'level_0'])

In [ ]:
# explicitly turning descriptions to type str

df_clean['Full Description'] = df_clean['Full Description'].apply(lambda row: str(row))

In [ ]:
# converting data frame column to a list called description_list

description_list = df_clean['Full Description'].tolist()

In [ ]:
# defining benchmark string

benchmark = 'Our AI-powered solutions address major challenges that are facing the healthcare field. Right now, the demand for diagnostic services is outpacing the supply of experts in the workforce. Developing solutions for managing this ever-increasing workload is a crucial task for the healthcare sector. Moreover, as the workload is growing, diagnostics and treatment are also becoming more complex. Diagnostic experts and physicians need a new set of tools that can handle large volumes of medical data quickly and accurately, allowing you to make more objective treatment decisions based on quantitative data and tailored to the needs of the individual patient. To provide this new toolset, we will need to draw on the power of artificial intelligence (AI).'

In [ ]:
# appending benchmark to description_list

description_list.append(str(benchmark))

In [ ]:
# defining a function called preprocessor, which takes a list of strings as input and returns a list of processed strings

def preprocessor(lst):
    processed_strings = [re.sub(r'\d+','', preprocessing.normalize_whitespace(preprocessing.remove_punctuation(lst[i].lower()))) for i in range(len(lst))]
    stopwords_removed = [remove_stopwords(processed_strings[i]) for i in range(len(processed_strings))]
    return stopwords_removed

In [ ]:
# processing strings to remove whitespace, punctuation, and numbers and converting case to lower

processed_strings_no_stopwords = preprocessor(description_list)

In [ ]:
# converting list of preprocessed strings to one-column data frame to prepare for lemmatization

processed_strings_df = pd.DataFrame(processed_strings_no_stopwords, columns=['strings'])

In [ ]:
# defining functions to perform lemmatization with appropriate POS tagging

lemmatizer = nltk.stem.WordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()

# converts nltk tags to wordnet tags
def nltk_tag_to_wordnet_tag(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

# takes a sentence string as input and returns the lemmatized sentence string
def lemmatize_sentence(sentence):
    nltk_tagged = nltk.pos_tag(nltk.word_tokenize(sentence))
    wordnet_tagged = map(lambda x: (x[0], nltk_tag_to_wordnet_tag(x[1])), nltk_tagged)
    lemmatized_sentence = []
    for word, tag in wordnet_tagged:
        if tag is None:
            lemmatized_sentence.append(word)
        else:
            lemmatized_sentence.append(lemmatizer.lemmatize(word, tag))
    return " ".join(lemmatized_sentence)

In [ ]:
# performing lemmatization and converting the result back into a list

processed_strings_df['lemmatized strings'] = processed_strings_df['strings'].apply(lambda x: lemmatize_sentence(x))
preprocessed_descriptions_final = processed_strings_df['lemmatized strings'].tolist()

## Doc2Vec Implementation

In [6]:
# defining the training_data as the preprocessed company descriptions minus the benchmark string

training_data = preprocessed_descriptions_final[:-1]

In [7]:
# tagging the training data

tagged_training_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(training_data)]

In [14]:
# creating a Doc2Vec model and building the model's vocabulary using the previously tagged training data

model = Doc2Vec(min_count=1, dm=1)
model.build_vocab(tagged_training_data)

In [ ]:
# training the Doc2Vec model on the data

model.train(tagged_training_data, total_examples=model.corpus_count, epochs=model.iter)

In [23]:
# identifying the similarities with the benchmark string using infer_vector to generate a vector based on thereon

test_data = word_tokenize(preprocessed_descriptions_final[-1])
v1 = model.infer_vector(test_data)
similar_doc = model.docvecs.most_similar([v1], topn=len(model.docvecs))

In [ ]:
# retrieving the top 20 similar companies and storing them in a list

top20 = similar_doc[0:20]

similar_companies_index = [top20[i][0] for i in range(len(top20))]

In [ ]:
# printing top 20 similar results using Doc2Vec

for i in similar_companies_index:
    print(df_clean['Organization Name'][int(i)])
    print(df_clean['Full Description'][int(i)])
    print(df_clean['Categories'][int(i)])
    print("\n\n======================\n\n")